In [1]:
!pip install -q sentence-transformers scikit-learn datasets


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install google-generativeai


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install transformers[torch]


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [130]:
!pip install evaluate


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import google.generativeai as genai
import pandas as pd
import json
# from google.colab import userdata
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer



c:\Users\ADMIN\Documents\HP\old ssd\MY FILES\projects\ticket_classification\my_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import tqdm as notebook_tqdm

In [4]:
import numpy as np
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import torch

In [7]:

# # Set up Gemini API
# genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))



# generation_config = {
#   "temperature": 1,
#   "top_p": 0.95,
#   "top_k": 40,
#   "max_output_tokens": 8192,
#   "response_mime_type": "application/json",
# }

# model = genai.GenerativeModel(
#   model_name="gemini-1.5-flash-8b",
#   generation_config=generation_config,
# )

# # Define ticket categories
# categories = ["Technical Issue", "Billing Question", "Feature Request", "General Inquiry"]

# tickets = []
# for category in categories:
#     prompt = f"""Generate 50 realistic customer support tickets for the category '{category}'.
#     Format the output as a JSON array where each ticket has 'title' and 'description' fields.
#     Make the tickets diverse and realistic."""
#     try:
#         chat_session = model.start_chat()
#         response = chat_session.send_message(prompt)
#         generated_tickets = json.loads(response.text)

#         for ticket in generated_tickets:
#             ticket['category'] = category
#             tickets.append(ticket)

#     except json.JSONDecodeError as e:
#         print(f"Failed to parse JSON response for {category}: {e}")
#         print(f"Raw response: {response.text}")


# # Save to DataFrame
# df = pd.DataFrame(tickets)
# df.to_csv("training_tickets.csv", index=False)

# print("Tickets saved to training_tickets.csv")

In [5]:
df = pd.read_csv('merged_new.csv')

In [6]:
df.shape

(928, 4)

In [7]:
# Create a new DataFrame without duplicates in 'description'
df = df.drop_duplicates(subset='description', keep='first')

# Reset the index of the new DataFrame
df.reset_index(drop=True, inplace=True)

In [8]:
print(df['category'].value_counts())

category
Feature Request     248
Billing Question    231
General Inquiry     230
Technical Issue     212
Name: count, dtype: int64


In [9]:
paraphrase_df = df[['title', 'paraphased_descripton', 'category']].copy()

In [10]:
paraphrase_df.rename(columns={'paraphased_descripton': 'description'}, inplace=True)

In [11]:
df_combined = pd.concat([df[['title', 'description', 'category']], paraphrase_df], ignore_index=True)

In [12]:
df = df_combined

In [13]:
# Save DataFrame to a CSV file
df.to_csv('final_tickets_dataset.csv', index=False)


In [110]:
df['category'].value_counts()

category
Feature Request     496
Billing Question    462
General Inquiry     460
Technical Issue     424
Name: count, dtype: int64

In [111]:
df.sample(10)

,title,description,category
1334,Policy Questions,User needs more information on the privacy pol...,General Inquiry
873,Dark mode theme,"Add a dark mode theme for improved usability, ...",Feature Request
1245,Tablet Won't Turn On,Tablet is not turning on after several power b...,Technical Issue
614,Email Delivery Problems,Emails are not being delivered to customer inb...,Technical Issue
1799,Offline Data Synchronization,Enable offline data synchronization.,Feature Request
1156,Offline Mode Functionality for the App,"Allow users to access key app functions, like ...",Feature Request
509,Printer Offline,My printer is offline and not responding. I've...,Technical Issue
993,Installation Instructions,User requesting installation instructions for ...,General Inquiry
901,Contact Options,What are the different ways I can contact cust...,General Inquiry
512,Payment Processing Issue,"I'm trying to make a payment, but the payment ...",Technical Issue


In [112]:
label_list = df["category"].unique().tolist()
label_to_id = {label: i for i, label in enumerate(label_list)}
df["label"] = df["category"].map(label_to_id)


In [113]:
train_df, test_df = train_test_split(df, test_size=0.2,stratify=df['category'], random_state=42)


In [114]:
train_df['category'].value_counts()

category
Feature Request     397
Billing Question    369
General Inquiry     368
Technical Issue     339
Name: count, dtype: int64

In [115]:
df["text"] = df["description"]

In [116]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [117]:
train_dataset

Dataset({
    features: ['title', 'description', 'category', 'label', '__index_level_0__'],
    num_rows: 1473
})

In [118]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [123]:
def tokenize_function(examples):
    return tokenizer(examples["description"], padding="max_length", truncation=True, max_length=128)


In [124]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1473 [00:00<?, ? examples/s]

Map: 100%|██████████| 369/369 [00:00<00:00, 7429.85 examples/s]


In [125]:
# Set format for PyTorch
columns = ["input_ids", "attention_mask", "label"]
train_dataset.set_format(type="torch", columns=columns)
test_dataset.set_format(type="torch", columns=columns)


In [126]:
num_labels = len(label_to_id)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=num_labels
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [127]:
from transformers import EarlyStoppingCallback


In [132]:
import evaluate

In [133]:
accuracy_metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)


In [143]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Training arguments
training_args = TrainingArguments(
    output_dir="./ticket_classifier_1",
    num_train_epochs=10,
    per_device_train_batch_size=2,  # Small batch size for this tiny dataset
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=1,
    learning_rate=0.001,
    weight_decay=0.01,
    load_best_model_at_end=True,  # To load the best model when training ends
    metric_for_best_model="eval_loss",  # Use eval_loss to track the best model
    greater_is_better=False,  # eval_loss is minimized, so False
)

# Trainer with EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Stop after 2 epochs of no improvement
)


c:\Users\ADMIN\Documents\HP\old ssd\MY FILES\projects\ticket_classification\my_env\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14308\1877148255.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [144]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.495600,1.385095,0.268293


KeyboardInterrupt: 

In [136]:
results = trainer.evaluate()
print("Evaluation results:", results)


Evaluation results: {'eval_loss': 0.24765925109386444, 'eval_accuracy': 0.948509485094851, 'eval_runtime': 16.0084, 'eval_samples_per_second': 23.05, 'eval_steps_per_second': 11.556, 'epoch': 3.0}


In [138]:
model_save_path = "./ticket_classifier_model_0.01lr"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print("Model saved to:", model_save_path)


Model saved to: ./ticket_classifier_model_10


In [8]:
label_list = df["category"].unique().tolist()
label_to_id = {label: i for i, label in enumerate(label_list)}
df["label"] = df["category"].map(label_to_id)


In [9]:
id_to_label = {v: k for k, v in label_to_id.items()}

print(id_to_label)  # Output: {0: 'Billing Question'}


{0: 'Billing Question', 1: 'Feature Request', 2: 'General Inquiry', 3: 'Technical Issue'}


In [10]:
id_to_label = {0: 'Billing Question', 1: 'Feature Request', 2: 'General Inquiry', 3: 'Technical Issue'}

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the fine-tuned model and tokenizer from the saved directory
model_path = "./ticket_classifier_model_10"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Example new ticket data (replace or extend these strings with your new tickets)
new_texts = [
    "I was charged $75 for a service that should have been free. Please check my account.",       # Expected: Billing Question
    "Could you add an option to integrate with my calendar?", # Expected: Feature Request
    "What are your support hours? I need to know when I can get assistance.",                         # Expected: General Inquiry
    "My application crashes every time I try to upload a file."      # Expected: Technical Issue
]
# Tokenize the new texts with the same parameters used during training
inputs = tokenizer(new_texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

# Switch the model to evaluation mode
model.eval()

# Run inference (no_grad disables gradient computation for efficiency)
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    # Get the index of the highest logit for each ticket
    predictions = torch.argmax(logits, dim=1).tolist()

predicted_labels = [id_to_label[pred] for pred in predictions]

print("Predicted ticket classes:")
for text, label in zip(new_texts, predicted_labels):
    print(f"Ticket: {text}\nPredicted class: {label}\n")


Predicted ticket classes:
Ticket: I was charged $75 for a service that should have been free. Please check my account.
Predicted class: Billing Question

Ticket: Could you add an option to integrate with my calendar?
Predicted class: General Inquiry

Ticket: What are your support hours? I need to know when I can get assistance.
Predicted class: General Inquiry

Ticket: My application crashes every time I try to upload a file.
Predicted class: Technical Issue



In [15]:
!pip install -U huggingface_hub[cli]



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
model.push_to_hub("Ticket-classification-model_v2")

model.safetensors: 100%|██████████| 268M/268M [00:30<00:00, 8.87MB/s] 


CommitInfo(commit_url='https://huggingface.co/Dragneel/Ticket-classification-model_v2/commit/c0a8077e1391894a1ad9c761fe8bf291f9310a3e', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='c0a8077e1391894a1ad9c761fe8bf291f9310a3e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Dragneel/Ticket-classification-model_v2', endpoint='https://huggingface.co', repo_type='model', repo_id='Dragneel/Ticket-classification-model_v2'), pr_revision=None, pr_num=None)